In [45]:
import sp_model 
from util import to_range, DATA_PATH, FIG_PATH, OptimizationMethod

w = 0.1


# %%
# 有一個趨勢是幾乎每個點都會開立一個rdc, cs，很像沒有不開的。
m, obj1, obj2 = sp_model.solve(1, OptimizationMethod.WEIGHTED_SUM)
obj1_star = obj1
m, obj1, obj2 = sp_model.solve(0, OptimizationMethod.WEIGHTED_SUM)
obj2_star = obj2
objstars = [obj1_star, obj2_star]
spm, o1, o2 = sp_model.solve(w,
                OptimizationMethod.LP_METRIC,
                objstars)

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 4636 rows, 4452 columns and 13080 nonzeros
Model fingerprint: 0xa75f41df
Model has 666 quadratic constraints
Model has 12 general constraints
Variable types: 4422 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+13]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [3e-01, 1e+00]
  Objective range  [2e-03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
  QRHS range       [7e+00, 2e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---

In [46]:
# get supplier names 
import pandas as pd 
PATH_PREFIX = 'MoDRL_'
df_supplier = pd.read_csv(DATA_PATH 
+ f'/{PATH_PREFIX}supplier.csv')
suppliers = df_supplier['Suppliers'].values 

df_distance = pd.read_csv(DATA_PATH + f'/{PATH_PREFIX}distance.csv')
AA = df_distance.columns.values
print(suppliers)
print(AA)
Commodities = ['Water', 'Food', 'Shelter']

['Sari' 'Qazvin' 'Tehran' 'Arak' 'Isfahan']
['Gorgan' 'Semnan' 'Sari' 'Rasht' 'Qazvin' 'Karaj' 'Tehran' 'Varamin'
 'Robatkarim' 'Islamshahr' 'Shahriar' 'Gom' 'Arak' 'Isfahan' 'Kashan']


### $\alpha_j, \beta_j$ Build RDC/CS or not 

In [41]:
from sp_model import SET, DELTA, PARAMETER
alpha, beta = [], [] 
for j in to_range(SET['J']):
    alpha_j = spm.getVarByName(f'alpha[{j}]').x
    beta_j = spm.getVarByName(f'beta[{j}]').x
    # alpha_i: build RDC or not 
    # beta_i: build CS or not
    alpha.append(alpha_j)
    beta.append(beta_j)
    # print(AA[i], alpha_i, beta_i)
build_df = pd.DataFrame({'RDC': alpha, 'CS': beta}, index=AA)
build_df = build_df.transpose() 
build_df

,Gorgan,Semnan,Sari,Rasht,Qazvin,Karaj,Tehran,Varamin,Robatkarim,Islamshahr,Shahriar,Gom,Arak,Isfahan,Kashan
RDC,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
CS,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,-0.0


In [47]:
i, j, k, k_prime, s, c = [len(idx) for idx in SET.values()]
import numpy as np 
Q = np.zeros((i, j, c))
X = np.zeros((i, j, c, s))
Y = np.zeros((j, k, c, s))

### $Q_{ijc}, X_{ijcs}$ transported commodities to RDC/CS

In [48]:
## get Q_ijc: Sup to RDC/Cs in preparedness phase 
for i in to_range(SET['I']):
    for j in to_range(SET['J']):
        for c in to_range(SET['C']):
            Q[i, j, c] = spm.getVarByName(f'Q[{i},{j},{c}]').x
# get X_ijcs: Sup to RDC/Cs in response phase 
for s in to_range(SET['S']):
    print('s=', s)
    for i in to_range(SET['I']):
        for j in to_range(SET['J']):
            for c in to_range(SET['C']):
                # print(i, j, c, s)
                X[i, j, c, s] = spm.getVarByName(f'X[{i},{j},{c},{s}]').x
                if X[i, j, c, s] != 0:
                    print(f'{suppliers[i]} => {AA[j]}', Commodities[c], X[i, j, c, s])



s= 0
Sari => Sari Food 117.0
Qazvin => Tehran Food 16.599999999999966
Qazvin => Shahriar Food 100.40000000000003
Tehran => Tehran Water 55.0
Tehran => Tehran Food 132.6
Arak => Tehran Food 117.0
s= 1
Sari => Sari Water 117.0
Sari => Sari Food 117.0
Qazvin => Shahriar Water 117.0
Qazvin => Shahriar Food 117.0
Tehran => Islamshahr Water 132.6
Tehran => Islamshahr Food 132.6
Tehran => Islamshahr Shelter 44.2
Arak => Islamshahr Water 117.0
Arak => Islamshahr Food 117.0
Arak => Islamshahr Shelter 39.0
Isfahan => Isfahan Food 117.0
s= 2
Sari => Sari Water 117.0
Sari => Sari Food 117.0
Sari => Sari Shelter 39.0
Qazvin => Shahriar Water 117.0
Qazvin => Shahriar Food 117.0
Qazvin => Shahriar Shelter 39.0
Tehran => Islamshahr Water 132.6
Tehran => Islamshahr Food 132.6
Tehran => Islamshahr Shelter 44.2
Arak => Islamshahr Water 117.0
Arak => Islamshahr Food 117.0
Arak => Islamshahr Shelter 39.0
Isfahan => Isfahan Water 117.0
Isfahan => Isfahan Shelter 39.0
Isfahan => Kashan Food 117.0
s= 3
Sari =

### $Y_{jkcs}$ transported commodities to AA

In [49]:
    
def isNotZero(x):
    return abs(x) > 1e-6
for s in to_range(SET['S']):
    print('s=', s)
    for j in to_range(SET['J']):
        for k in to_range(SET['K']):
            for c in to_range(SET['C']):
                Y[j, k, c, s] = spm.getVarByName(f'Y[{j},{k},{c},{s}]').x
                identity = 'RDC' if alpha[j] == 1 else ''
                identity = 'CS' if beta[j] == 1 else '' 
                if isNotZero(Y[j, k, c, s]):
                    # J and K share the same node set 
                    print(f'{identity} {AA[j]} => {AA[k]}', Commodities[c], Y[j, k, c, s])

s= 0
 Sari => Gorgan Food 6.199999999999989
 Sari => Rasht Food 110.80000000000001
 Tehran => Sari Water 55.0
 Tehran => Sari Food 266.19999999999993
 Shahriar => Rasht Food 100.40000000000003
s= 1
 Sari => Tehran Water 117.0
 Sari => Tehran Food 117.0
 Islamshahr => Tehran Water 249.6
 Islamshahr => Tehran Food 249.6
 Islamshahr => Tehran Shelter 83.2
 Shahriar => Tehran Water 117.0
 Shahriar => Tehran Food 117.0
 Isfahan => Tehran Food 117.0
s= 2
 Sari => Tehran Water 117.0
 Sari => Tehran Food 117.0
 Sari => Tehran Shelter 39.0
 Karaj => Arak Food 75.0
 Islamshahr => Tehran Water 249.6
 Islamshahr => Tehran Food 249.6
 Islamshahr => Tehran Shelter 83.2
 Shahriar => Tehran Water 117.0
 Shahriar => Tehran Food 117.0
 Shahriar => Tehran Shelter 39.0
 Arak => Robatkarim Food 100.0
 Isfahan => Tehran Water 117.0
 Isfahan => Tehran Shelter 39.0
 Kashan => Tehran Food 117.0
s= 3
 Sari => Tehran Water 117.0
 Sari => Tehran Food 117.0
 Sari => Tehran Shelter 39.0
 Islamshahr => Tehran Water 